# 실행환경 : 윈도우

# <span style="font-weight:bold">중고차 가격 예측을 위해 대표적인 중고차 사이트 Encar에서 정보 획득</span>
### <span style="font-weight:bold">가장 대중적이고 데이터 갯수가 많은 브랜드 현대를 선택, 그 중 인기가 많은 상위 차종 선별</span>
> <span style="color:blue ; font-weight:bold"> 제네시스는 현대에서 브랜드 분리되었으나 대중적이라 할 만큼 인기 차종들이 많아 열외로 선택</span>

## Encar 아반떼 자료 수집

In [ ]:
###### ENCAR 사이트에서 중고차 기본 정보 가져오기 - 아반떼 ######

import requests
import json
import time
import pandas as pd
from collections import Counter



#### 아반떼 기본 데이터 저장 ####
# Base URL 및 헤더 설정
base_url = "https://api.encar.com/search/car/list/premium"
headers = {
    "User-Agent": "Mozilla/5.0"
}

# 전체 데이터를 저장할 리스트
all_data = []
offset = 0
limit = 20

while True:
    # API 요청 파라미터 설정
    params = {
        "count": "true",
        "q": "(And.Hidden.N._.(C.CarType.Y._.(C.Manufacturer.현대._.ModelGroup.아반떼.)))",
        "sr": f"|ModifiedDate|{offset}|{limit}"
    }

    print(f"🚀 Fetching data from offset {offset}...")
    
    try:
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"🚨 요청 중 오류 발생: {e}")
        break
    except json.JSONDecodeError as e:
        print(f"🚨 JSON 파싱 오류 발생: {e}")
        break

    # SearchResults에서 데이터 추출
    results = data.get("SearchResults", [])
    
    if not results:  # 빈 데이터 처리 (마지막 페이지 도달)
        print("✅ 마지막 페이지 도달 또는 데이터 없음.")
        break

    # 전체 데이터 리스트에 추가
    all_data.extend(results)
    print(f"✅ Offset {offset}: {len(results)}건 추가 완료 (누적: {len(all_data)}건)")

    # 다음 offset으로 이동
    offset += limit

    # 서버 과부하 방지 (1초 대기)
    time.sleep(1)

# 결과를 JSON 파일로 저장
try:
    with open("아반떼_기본.json", "w", encoding="utf-8-sig") as f:
        json.dump(all_data, f, ensure_ascii=False, indent=4)
    print("✅ JSON 파일 저장 완료")
except Exception as e:
    print(f"🚨 파일 저장 중 오류 발생: {e}")

print(f"\n🎉 총 {len(all_data)}건의 아반떼 데이터 저장 완료!")







#### 저장된 json 파일에서 데이터 끌어와 데이터 프레임으로 만들기 ####

with open("아반떼_기본.json", "r", encoding="utf-8-sig") as file:
data = json.load(file)

# 필요한 정보만 추출
car_data = []
for car in data:
car_data.append({
"Id": car.get("Id", "N/A"),
"제조사": car.get("Manufacturer", "N/A"),
"모델명": car.get("Model", "N/A"),
"배지": car.get("Badge", "N/A"), # 트림 정보
"세부 배지": car.get("BadgeDetail", "N/A"), # 세부 트림
"연식": car.get("Year", "N/A"),
"모델연도": car.get("FormYear", "N/A"),
"가격": car.get("Price", "N/A"),
"주행거리": car.get("Mileage", "N/A"),
"연료": car.get("FuelType", "N/A"),
"변속기": car.get("Transmission", "N/A"),
"등록 상태": car.get("ServiceCopyCar", "N/A"), # 중복 차량 여부
"판매 유형": car.get("SellType", "N/A"),
"마지막 수정일": car.get("ModifiedDate", "N/A"),
"판매지역": car.get("OfficeCityState", "N/A"),
"판매처": car.get("OfficeName", "N/A"),
})

df = pd.DataFrame(car_data)


### 등록상태 DUPLICATION 날리기 ###
df_unique = df.loc[df['등록 상태']!='DUPLICATION']


### 기본 정보 안에서 Id 중복 제거 ###
df_unique = df_unique.drop_duplicates(subset='Id')
df_unique.to_csv('아반떼_기본_중복제거.csv', index=False, encoding='utf-8-sig')






#### 기존 데이터 중 Id로 다른 API를 통해 차량 번호 가져온 후 새로운 열로 추가 ####


# CSV 파일 불러오기
df_unique = pd.read_csv('아반떼_기본_중복제거.csv')

# API 기본 URL 설정
base_url = "https://api.encar.com/v1/readside/vehicle/{Id}?include=CONTENTS"

# vehicleNo 저장할 빈 리스트 생성
vehicle_numbers = []

# 각 Id에 대해 API 호출 및 JSON 저장하기
for idx, car_id in enumerate(df_unique['Id'], start=1):
url = base_url.format(Id=car_id)
try:
response = requests.get(url)
response.raise_for_status()
data = response.json()

# vehicleNo 추출
vehicle_no = data.get('vehicleNo', None)
vehicle_numbers.append(vehicle_no)
print(f"Processed {idx}/{len(df_unique)}: ID={car_id}")

except requests.exceptions.RequestException as e:
print(f"Error fetching data for ID {car_id}: {e}")
vehicle_numbers.append(None)

# 서버 과부하 방지를 위한 지연 추가
time.sleep(0.5)

# 원본 DataFrame에 vehicleNo 열 추가하기
df_unique['차량번호'] = vehicle_numbers

# 결과를 새로운 CSV 파일로 저장
df_unique.to_csv('아반떼_기본_중복제거_차량번호.csv', index=False, encoding='utf-8-sig')






#### Id와 차량번호 이용해 보험이력 데이터 가져와 json 파일로 저장 ####


# 차량 번호 추가한 CSV 파일 불러오기
df_unique_no = pd.read_csv('아반떼_기본_중복제거_차량번호.csv')
df_unique_no = df_unique_no.rename(columns={'vehicleNo':'차량번호'})
# API 기본 URL 설정
base_url = "https://api.encar.com/v1/readside/record/vehicle/{Id}/open?vehicleNo={CarNo}"

# JSON 데이터를 저장할 리스트 생성
all_data = []

# 각 Id와 vehicleNo에 대해 API 호출 및 데이터 수집
for idx, row in df_unique_no.iterrows():
car_id = row['Id']
vehicle_no = row['차량번호']
url = base_url.format(Id=car_id, CarNo=vehicle_no)

try:
response = requests.get(url)
response.raise_for_status()
data = response.json()

# JSON 데이터 리스트에 추가
all_data.append(data)

# 진행 상태 출력
print(f"Processed {idx + 1}/{len(df_unique_no)}: ID={car_id}, Vehicle No={vehicle_no}")

except requests.exceptions.RequestException as e:
print(f"Error fetching data for ID {car_id}, Vehicle No {vehicle_no}: {e}")

# 서버 과부하 방지를 위한 지연 추가
time.sleep(0.5)

# 모든 데이터를 하나의 JSON 파일로 저장
with open("아반떼_기본_중복제거_차량번호_보험이력.json", "w", encoding='utf-8-sig') as json_file:
json.dump(all_data, json_file, ensure_ascii=False, indent=4)

print("All data has been saved to '아반떼_기본_중복제거_차량번호_보험이력.json'")








#### json데이터에서 원하는 보험이력 정보 선택해 데이터프레임 만들기 ####

with open("아반떼_기본_중복제거_차량번호_보험이력.json", "r", encoding="utf-8-sig") as file:
data = json.load(file)

# 필요한 정보만 추출
insurance_data = []
for insurance in data:
insurance_data.append({
"차량번호": insurance.get("carNo", "N/A"),
"내차피해": insurance.get("myAccidentCnt", "N/A"),
"타차가해": insurance.get("otherAccidentCnt", "N/A"),
"소유자변경이력": insurance.get("ownerChangeCnt", "N/A"),
"내차피해금액": insurance.get("myAccidentCost", "N/A"),
"타차가해금액": insurance.get("otherAccidentCost", "N/A"),
"번호판변경이력": insurance.get("carNoChangeCnt", "N/A"),
"전체침수횟수": insurance.get("floodTotalLossCnt", "N/A"),
"일부침수횟수": insurance.get("floodPartLossCnt", "N/A"),
})
ins = pd.DataFrame(insurance_data)

#중복 제거
print(f"기존 행 수: {len(ins)}")
ins = ins.drop_duplicates()
print(f"중복 제거 후 행 수: {len(ins)}")


#기존 기본 정보의 금액 단위와 맞추기 위해 만원 단위로 변환
ins['내차피해금액'] = ins['내차피해금액']/10000
ins['타차가해금액'] = ins['타차가해금액']/10000


# csv 파일로 백업
ins.to_csv('아반떼_보험이력.csv', index=False, encoding='utf-8-sig')

# 차량번호 추가한 기본 정보에 보험 이력 추가
df_unique_no_ins = pd.merge(df_unique_no, ins, on='차량번호', how='inner')


df_unique_no_ins.to_csv('아반떼_기본_중복제거_차량번호_보험이력.csv', index=False, encoding='utf-8-sig')
print("All data has been saved to '아반떼_기본_중복제거_차량번호_보험이력.csv'")

## Encar 쏘나타 자료 수집

In [ ]:
###### ENCAR 사이트에서 중고차 기본 정보 가져오기 - 쏘나타 ######

import requests
import json
import time
import pandas as pd
from collections import Counter



#### 쏘나타 기본 데이터 저장 ####
# Base URL 및 헤더 설정
base_url = "https://api.encar.com/search/car/list/premium"
headers = {
    "User-Agent": "Mozilla/5.0"
}

# 전체 데이터를 저장할 리스트
all_data = []
offset = 0
limit = 20

while True:
    # API 요청 파라미터 설정
    params = {
        "count": "true",
        "q": "(And.Hidden.N._.(C.CarType.Y._.(C.Manufacturer.현대._.ModelGroup.쏘나타.)))",
        "sr": f"|ModifiedDate|{offset}|{limit}"
    }

    print(f"🚀 Fetching data from offset {offset}...")
    
    try:
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"🚨 요청 중 오류 발생: {e}")
        break
    except json.JSONDecodeError as e:
        print(f"🚨 JSON 파싱 오류 발생: {e}")
        break

    # SearchResults에서 데이터 추출
    results = data.get("SearchResults", [])
    
    if not results:  # 빈 데이터 처리 (마지막 페이지 도달)
        print("✅ 마지막 페이지 도달 또는 데이터 없음.")
        break

    # 전체 데이터 리스트에 추가
    all_data.extend(results)
    print(f"✅ Offset {offset}: {len(results)}건 추가 완료 (누적: {len(all_data)}건)")

    # 다음 offset으로 이동
    offset += limit

    # 서버 과부하 방지 (1초 대기)
    time.sleep(1)

# 결과를 JSON 파일로 저장
try:
    with open("쏘나타_기본.json", "w", encoding="utf-8-sig") as f:
        json.dump(all_data, f, ensure_ascii=False, indent=4)
    print("✅ JSON 파일 저장 완료")
except Exception as e:
    print(f"🚨 파일 저장 중 오류 발생: {e}")

print(f"\n🎉 총 {len(all_data)}건의 쏘나타 데이터 저장 완료!")







#### 저장된 json 파일에서 데이터 끌어와 데이터 프레임으로 만들기 ####

with open("쏘나타_기본.json", "r", encoding="utf-8-sig") as file:
data = json.load(file)

# 필요한 정보만 추출
car_data = []
for car in data:
car_data.append({
"Id": car.get("Id", "N/A"),
"제조사": car.get("Manufacturer", "N/A"),
"모델명": car.get("Model", "N/A"),
"배지": car.get("Badge", "N/A"), # 트림 정보
"세부 배지": car.get("BadgeDetail", "N/A"), # 세부 트림
"연식": car.get("Year", "N/A"),
"모델연도": car.get("FormYear", "N/A"),
"가격": car.get("Price", "N/A"),
"주행거리": car.get("Mileage", "N/A"),
"연료": car.get("FuelType", "N/A"),
"변속기": car.get("Transmission", "N/A"),
"등록 상태": car.get("ServiceCopyCar", "N/A"), # 중복 차량 여부
"판매 유형": car.get("SellType", "N/A"),
"마지막 수정일": car.get("ModifiedDate", "N/A"),
"판매지역": car.get("OfficeCityState", "N/A"),
"판매처": car.get("OfficeName", "N/A"),
})

df = pd.DataFrame(car_data)


### 등록상태 DUPLICATION 날리기 ###
df_unique = df.loc[df['등록 상태']!='DUPLICATION']


### 기본 정보 안에서 Id 중복 제거 ###
df_unique = df_unique.drop_duplicates(subset='Id')
df_unique.to_csv('쏘나타_기본_중복제거.csv', index=False, encoding='utf-8-sig')






#### 기존 데이터 중 Id로 다른 API를 통해 차량 번호 가져온 후 새로운 열로 추가 ####


# CSV 파일 불러오기
df_unique = pd.read_csv('쏘나타_기본_중복제거.csv')

# API 기본 URL 설정
base_url = "https://api.encar.com/v1/readside/vehicle/{Id}?include=CONTENTS"

# vehicleNo 저장할 빈 리스트 생성
vehicle_numbers = []

# 각 Id에 대해 API 호출 및 JSON 저장하기
for idx, car_id in enumerate(df_unique['Id'], start=1):
url = base_url.format(Id=car_id)
try:
response = requests.get(url)
response.raise_for_status()
data = response.json()

# vehicleNo 추출
vehicle_no = data.get('vehicleNo', None)
vehicle_numbers.append(vehicle_no)
print(f"Processed {idx}/{len(df_unique)}: ID={car_id}")

except requests.exceptions.RequestException as e:
print(f"Error fetching data for ID {car_id}: {e}")
vehicle_numbers.append(None)

# 서버 과부하 방지를 위한 지연 추가
time.sleep(0.5)

# 원본 DataFrame에 vehicleNo 열 추가하기
df_unique['차량번호'] = vehicle_numbers

# 결과를 새로운 CSV 파일로 저장
df_unique.to_csv('쏘나타_기본_중복제거_차량번호.csv', index=False, encoding='utf-8-sig')






#### Id와 차량번호 이용해 보험이력 데이터 가져와 json 파일로 저장 ####


# 차량 번호 추가한 CSV 파일 불러오기
df_unique_no = pd.read_csv('쏘나타_기본_중복제거_차량번호.csv')
df_unique_no = df_unique_no.rename(columns={'vehicleNo':'차량번호'})
# API 기본 URL 설정
base_url = "https://api.encar.com/v1/readside/record/vehicle/{Id}/open?vehicleNo={CarNo}"

# JSON 데이터를 저장할 리스트 생성
all_data = []

# 각 Id와 vehicleNo에 대해 API 호출 및 데이터 수집
for idx, row in df_unique_no.iterrows():
car_id = row['Id']
vehicle_no = row['차량번호']
url = base_url.format(Id=car_id, CarNo=vehicle_no)

try:
response = requests.get(url)
response.raise_for_status()
data = response.json()

# JSON 데이터 리스트에 추가
all_data.append(data)

# 진행 상태 출력
print(f"Processed {idx + 1}/{len(df_unique_no)}: ID={car_id}, Vehicle No={vehicle_no}")

except requests.exceptions.RequestException as e:
print(f"Error fetching data for ID {car_id}, Vehicle No {vehicle_no}: {e}")

# 서버 과부하 방지를 위한 지연 추가
time.sleep(0.5)

# 모든 데이터를 하나의 JSON 파일로 저장
with open("쏘나타_기본_중복제거_차량번호_보험이력.json", "w", encoding='utf-8-sig') as json_file:
json.dump(all_data, json_file, ensure_ascii=False, indent=4)

print("All data has been saved to '쏘나타_기본_중복제거_차량번호_보험이력.json'")








#### json데이터에서 원하는 보험이력 정보 선택해 데이터프레임 만들기 ####

with open("쏘나타_기본_중복제거_차량번호_보험이력.json", "r", encoding="utf-8-sig") as file:
data = json.load(file)

# 필요한 정보만 추출
insurance_data = []
for insurance in data:
insurance_data.append({
"차량번호": insurance.get("carNo", "N/A"),
"내차피해": insurance.get("myAccidentCnt", "N/A"),
"타차가해": insurance.get("otherAccidentCnt", "N/A"),
"소유자변경이력": insurance.get("ownerChangeCnt", "N/A"),
"내차피해금액": insurance.get("myAccidentCost", "N/A"),
"타차가해금액": insurance.get("otherAccidentCost", "N/A"),
"번호판변경이력": insurance.get("carNoChangeCnt", "N/A"),
"전체침수횟수": insurance.get("floodTotalLossCnt", "N/A"),
"일부침수횟수": insurance.get("floodPartLossCnt", "N/A"),
})
ins = pd.DataFrame(insurance_data)

#중복 제거
print(f"기존 행 수: {len(ins)}")
ins = ins.drop_duplicates()
print(f"중복 제거 후 행 수: {len(ins)}")


#기존 기본 정보의 금액 단위와 맞추기 위해 만원 단위로 변환
ins['내차피해금액'] = ins['내차피해금액']/10000
ins['타차가해금액'] = ins['타차가해금액']/10000


# csv 파일로 백업
ins.to_csv('쏘나타_보험이력.csv', index=False, encoding='utf-8-sig')

# 차량번호 추가한 기본 정보에 보험 이력 추가
df_unique_no_ins = pd.merge(df_unique_no, ins, on='차량번호', how='inner')


df_unique_no_ins.to_csv('쏘나타_기본_중복제거_차량번호_보험이력.csv', index=False, encoding='utf-8-sig')
print("All data has been saved to '쏘나타_기본_중복제거_차량번호_보험이력.csv'")

## Encar 투싼 자료 수집

In [ ]:
###### ENCAR 사이트에서 중고차 기본 정보 가져오기 - 투싼 ######

import requests
import json
import time
import pandas as pd
from collections import Counter



#### 투싼 기본 데이터 저장 ####
# Base URL 및 헤더 설정
base_url = "https://api.encar.com/search/car/list/premium"
headers = {
    "User-Agent": "Mozilla/5.0"
}

# 전체 데이터를 저장할 리스트
all_data = []
offset = 0
limit = 20

while True:
    # API 요청 파라미터 설정
    params = {
        "count": "true",
        "q": "(And.Hidden.N._.(C.CarType.Y._.(C.Manufacturer.현대._.ModelGroup.투싼.)))",
        "sr": f"|ModifiedDate|{offset}|{limit}"
    }

    print(f"🚀 Fetching data from offset {offset}...")
    
    try:
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"🚨 요청 중 오류 발생: {e}")
        break
    except json.JSONDecodeError as e:
        print(f"🚨 JSON 파싱 오류 발생: {e}")
        break

    # SearchResults에서 데이터 추출
    results = data.get("SearchResults", [])
    
    if not results:  # 빈 데이터 처리 (마지막 페이지 도달)
        print("✅ 마지막 페이지 도달 또는 데이터 없음.")
        break

    # 전체 데이터 리스트에 추가
    all_data.extend(results)
    print(f"✅ Offset {offset}: {len(results)}건 추가 완료 (누적: {len(all_data)}건)")

    # 다음 offset으로 이동
    offset += limit

    # 서버 과부하 방지 (1초 대기)
    time.sleep(1)

# 결과를 JSON 파일로 저장
try:
    with open("투싼_기본.json", "w", encoding="utf-8-sig") as f:
        json.dump(all_data, f, ensure_ascii=False, indent=4)
    print("✅ JSON 파일 저장 완료")
except Exception as e:
    print(f"🚨 파일 저장 중 오류 발생: {e}")

print(f"\n🎉 총 {len(all_data)}건의 투싼 데이터 저장 완료!")







#### 저장된 json 파일에서 데이터 끌어와 데이터 프레임으로 만들기 ####

with open("투싼_기본.json", "r", encoding="utf-8-sig") as file:
data = json.load(file)

# 필요한 정보만 추출
car_data = []
for car in data:
car_data.append({
"Id": car.get("Id", "N/A"),
"제조사": car.get("Manufacturer", "N/A"),
"모델명": car.get("Model", "N/A"),
"배지": car.get("Badge", "N/A"), # 트림 정보
"세부 배지": car.get("BadgeDetail", "N/A"), # 세부 트림
"연식": car.get("Year", "N/A"),
"모델연도": car.get("FormYear", "N/A"),
"가격": car.get("Price", "N/A"),
"주행거리": car.get("Mileage", "N/A"),
"연료": car.get("FuelType", "N/A"),
"변속기": car.get("Transmission", "N/A"),
"등록 상태": car.get("ServiceCopyCar", "N/A"), # 중복 차량 여부
"판매 유형": car.get("SellType", "N/A"),
"마지막 수정일": car.get("ModifiedDate", "N/A"),
"판매지역": car.get("OfficeCityState", "N/A"),
"판매처": car.get("OfficeName", "N/A"),
})

df = pd.DataFrame(car_data)


### 등록상태 DUPLICATION 날리기 ###
df_unique = df.loc[df['등록 상태']!='DUPLICATION']


### 기본 정보 안에서 Id 중복 제거 ###
df_unique = df_unique.drop_duplicates(subset='Id')
df_unique.to_csv('투싼_기본_중복제거.csv', index=False, encoding='utf-8-sig')






#### 기존 데이터 중 Id로 다른 API를 통해 차량 번호 가져온 후 새로운 열로 추가 ####


# CSV 파일 불러오기
df_unique = pd.read_csv('투싼_기본_중복제거.csv')

# API 기본 URL 설정
base_url = "https://api.encar.com/v1/readside/vehicle/{Id}?include=CONTENTS"

# vehicleNo 저장할 빈 리스트 생성
vehicle_numbers = []

# 각 Id에 대해 API 호출 및 JSON 저장하기
for idx, car_id in enumerate(df_unique['Id'], start=1):
url = base_url.format(Id=car_id)
try:
response = requests.get(url)
response.raise_for_status()
data = response.json()

# vehicleNo 추출
vehicle_no = data.get('vehicleNo', None)
vehicle_numbers.append(vehicle_no)
print(f"Processed {idx}/{len(df_unique)}: ID={car_id}")

except requests.exceptions.RequestException as e:
print(f"Error fetching data for ID {car_id}: {e}")
vehicle_numbers.append(None)

# 서버 과부하 방지를 위한 지연 추가
time.sleep(0.5)

# 원본 DataFrame에 vehicleNo 열 추가하기
df_unique['차량번호'] = vehicle_numbers

# 결과를 새로운 CSV 파일로 저장
df_unique.to_csv('투싼_기본_중복제거_차량번호.csv', index=False, encoding='utf-8-sig')






#### Id와 차량번호 이용해 보험이력 데이터 가져와 json 파일로 저장 ####


# 차량 번호 추가한 CSV 파일 불러오기
df_unique_no = pd.read_csv('투싼_기본_중복제거_차량번호.csv')
df_unique_no = df_unique_no.rename(columns={'vehicleNo':'차량번호'})
# API 기본 URL 설정
base_url = "https://api.encar.com/v1/readside/record/vehicle/{Id}/open?vehicleNo={CarNo}"

# JSON 데이터를 저장할 리스트 생성
all_data = []

# 각 Id와 vehicleNo에 대해 API 호출 및 데이터 수집
for idx, row in df_unique_no.iterrows():
car_id = row['Id']
vehicle_no = row['차량번호']
url = base_url.format(Id=car_id, CarNo=vehicle_no)

try:
response = requests.get(url)
response.raise_for_status()
data = response.json()

# JSON 데이터 리스트에 추가
all_data.append(data)

# 진행 상태 출력
print(f"Processed {idx + 1}/{len(df_unique_no)}: ID={car_id}, Vehicle No={vehicle_no}")

except requests.exceptions.RequestException as e:
print(f"Error fetching data for ID {car_id}, Vehicle No {vehicle_no}: {e}")

# 서버 과부하 방지를 위한 지연 추가
time.sleep(0.5)

# 모든 데이터를 하나의 JSON 파일로 저장
with open("투싼_기본_중복제거_차량번호_보험이력.json", "w", encoding='utf-8-sig') as json_file:
json.dump(all_data, json_file, ensure_ascii=False, indent=4)

print("All data has been saved to '투싼_기본_중복제거_차량번호_보험이력.json'")








#### json데이터에서 원하는 보험이력 정보 선택해 데이터프레임 만들기 ####

with open("투싼_기본_중복제거_차량번호_보험이력.json", "r", encoding="utf-8-sig") as file:
data = json.load(file)

# 필요한 정보만 추출
insurance_data = []
for insurance in data:
insurance_data.append({
"차량번호": insurance.get("carNo", "N/A"),
"내차피해": insurance.get("myAccidentCnt", "N/A"),
"타차가해": insurance.get("otherAccidentCnt", "N/A"),
"소유자변경이력": insurance.get("ownerChangeCnt", "N/A"),
"내차피해금액": insurance.get("myAccidentCost", "N/A"),
"타차가해금액": insurance.get("otherAccidentCost", "N/A"),
"번호판변경이력": insurance.get("carNoChangeCnt", "N/A"),
"전체침수횟수": insurance.get("floodTotalLossCnt", "N/A"),
"일부침수횟수": insurance.get("floodPartLossCnt", "N/A"),
})
ins = pd.DataFrame(insurance_data)

#중복 제거
print(f"기존 행 수: {len(ins)}")
ins = ins.drop_duplicates()
print(f"중복 제거 후 행 수: {len(ins)}")


#기존 기본 정보의 금액 단위와 맞추기 위해 만원 단위로 변환
ins['내차피해금액'] = ins['내차피해금액']/10000
ins['타차가해금액'] = ins['타차가해금액']/10000


# csv 파일로 백업
ins.to_csv('투싼_보험이력.csv', index=False, encoding='utf-8-sig')

# 차량번호 추가한 기본 정보에 보험 이력 추가
df_unique_no_ins = pd.merge(df_unique_no, ins, on='차량번호', how='inner')


df_unique_no_ins.to_csv('투싼_기본_중복제거_차량번호_보험이력.csv', index=False, encoding='utf-8-sig')
print("All data has been saved to '투싼_기본_중복제거_차량번호_보험이력.csv'")

## Encar 팰리세이드 자료 수집

In [ ]:
###### ENCAR 사이트에서 중고차 기본 정보 가져오기 - 팰리세이드 ######

import requests
import json
import time
import pandas as pd
from collections import Counter



#### 팰리세이드 기본 데이터 저장 ####
# Base URL 및 헤더 설정
base_url = "https://api.encar.com/search/car/list/premium"
headers = {
    "User-Agent": "Mozilla/5.0"
}

# 전체 데이터를 저장할 리스트
all_data = []
offset = 0
limit = 20

while True:
    # API 요청 파라미터 설정
    params = {
        "count": "true",
        "q": "(And.Hidden.N._.(C.CarType.Y._.(C.Manufacturer.현대._.ModelGroup.팰리세이드.)))",
        "sr": f"|ModifiedDate|{offset}|{limit}"
    }

    print(f"🚀 Fetching data from offset {offset}...")
    
    try:
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"🚨 요청 중 오류 발생: {e}")
        break
    except json.JSONDecodeError as e:
        print(f"🚨 JSON 파싱 오류 발생: {e}")
        break

    # SearchResults에서 데이터 추출
    results = data.get("SearchResults", [])
    
    if not results:  # 빈 데이터 처리 (마지막 페이지 도달)
        print("✅ 마지막 페이지 도달 또는 데이터 없음.")
        break

    # 전체 데이터 리스트에 추가
    all_data.extend(results)
    print(f"✅ Offset {offset}: {len(results)}건 추가 완료 (누적: {len(all_data)}건)")

    # 다음 offset으로 이동
    offset += limit

    # 서버 과부하 방지 (1초 대기)
    time.sleep(1)

# 결과를 JSON 파일로 저장
try:
    with open("팰리세이드_기본.json", "w", encoding="utf-8-sig") as f:
        json.dump(all_data, f, ensure_ascii=False, indent=4)
    print("✅ JSON 파일 저장 완료")
except Exception as e:
    print(f"🚨 파일 저장 중 오류 발생: {e}")

print(f"\n🎉 총 {len(all_data)}건의 팰리세이드 데이터 저장 완료!")







#### 저장된 json 파일에서 데이터 끌어와 데이터 프레임으로 만들기 ####

with open("팰리세이드_기본.json", "r", encoding="utf-8-sig") as file:
data = json.load(file)

# 필요한 정보만 추출
car_data = []
for car in data:
car_data.append({
"Id": car.get("Id", "N/A"),
"제조사": car.get("Manufacturer", "N/A"),
"모델명": car.get("Model", "N/A"),
"배지": car.get("Badge", "N/A"), # 트림 정보
"세부 배지": car.get("BadgeDetail", "N/A"), # 세부 트림
"연식": car.get("Year", "N/A"),
"모델연도": car.get("FormYear", "N/A"),
"가격": car.get("Price", "N/A"),
"주행거리": car.get("Mileage", "N/A"),
"연료": car.get("FuelType", "N/A"),
"변속기": car.get("Transmission", "N/A"),
"등록 상태": car.get("ServiceCopyCar", "N/A"), # 중복 차량 여부
"판매 유형": car.get("SellType", "N/A"),
"마지막 수정일": car.get("ModifiedDate", "N/A"),
"판매지역": car.get("OfficeCityState", "N/A"),
"판매처": car.get("OfficeName", "N/A"),
})

df = pd.DataFrame(car_data)


### 등록상태 DUPLICATION 날리기 ###
df_unique = df.loc[df['등록 상태']!='DUPLICATION']


### 기본 정보 안에서 Id 중복 제거 ###
df_unique = df_unique.drop_duplicates(subset='Id')
df_unique.to_csv('팰리세이드_기본_중복제거.csv', index=False, encoding='utf-8-sig')






#### 기존 데이터 중 Id로 다른 API를 통해 차량 번호 가져온 후 새로운 열로 추가 ####


# CSV 파일 불러오기
df_unique = pd.read_csv('팰리세이드_기본_중복제거.csv')

# API 기본 URL 설정
base_url = "https://api.encar.com/v1/readside/vehicle/{Id}?include=CONTENTS"

# vehicleNo 저장할 빈 리스트 생성
vehicle_numbers = []

# 각 Id에 대해 API 호출 및 JSON 저장하기
for idx, car_id in enumerate(df_unique['Id'], start=1):
url = base_url.format(Id=car_id)
try:
response = requests.get(url)
response.raise_for_status()
data = response.json()

# vehicleNo 추출
vehicle_no = data.get('vehicleNo', None)
vehicle_numbers.append(vehicle_no)
print(f"Processed {idx}/{len(df_unique)}: ID={car_id}")

except requests.exceptions.RequestException as e:
print(f"Error fetching data for ID {car_id}: {e}")
vehicle_numbers.append(None)

# 서버 과부하 방지를 위한 지연 추가
time.sleep(0.5)

# 원본 DataFrame에 vehicleNo 열 추가하기
df_unique['차량번호'] = vehicle_numbers

# 결과를 새로운 CSV 파일로 저장
df_unique.to_csv('팰리세이드_기본_중복제거_차량번호.csv', index=False, encoding='utf-8-sig')






#### Id와 차량번호 이용해 보험이력 데이터 가져와 json 파일로 저장 ####


# 차량 번호 추가한 CSV 파일 불러오기
df_unique_no = pd.read_csv('팰리세이드_기본_중복제거_차량번호.csv')
df_unique_no = df_unique_no.rename(columns={'vehicleNo':'차량번호'})
# API 기본 URL 설정
base_url = "https://api.encar.com/v1/readside/record/vehicle/{Id}/open?vehicleNo={CarNo}"

# JSON 데이터를 저장할 리스트 생성
all_data = []

# 각 Id와 vehicleNo에 대해 API 호출 및 데이터 수집
for idx, row in df_unique_no.iterrows():
car_id = row['Id']
vehicle_no = row['차량번호']
url = base_url.format(Id=car_id, CarNo=vehicle_no)

try:
response = requests.get(url)
response.raise_for_status()
data = response.json()

# JSON 데이터 리스트에 추가
all_data.append(data)

# 진행 상태 출력
print(f"Processed {idx + 1}/{len(df_unique_no)}: ID={car_id}, Vehicle No={vehicle_no}")

except requests.exceptions.RequestException as e:
print(f"Error fetching data for ID {car_id}, Vehicle No {vehicle_no}: {e}")

# 서버 과부하 방지를 위한 지연 추가
time.sleep(0.5)

# 모든 데이터를 하나의 JSON 파일로 저장
with open("팰리세이드_기본_중복제거_차량번호_보험이력.json", "w", encoding='utf-8-sig') as json_file:
json.dump(all_data, json_file, ensure_ascii=False, indent=4)

print("All data has been saved to '팰리세이드_기본_중복제거_차량번호_보험이력.json'")








#### json데이터에서 원하는 보험이력 정보 선택해 데이터프레임 만들기 ####

with open("팰리세이드_기본_중복제거_차량번호_보험이력.json", "r", encoding="utf-8-sig") as file:
data = json.load(file)

# 필요한 정보만 추출
insurance_data = []
for insurance in data:
insurance_data.append({
"차량번호": insurance.get("carNo", "N/A"),
"내차피해": insurance.get("myAccidentCnt", "N/A"),
"타차가해": insurance.get("otherAccidentCnt", "N/A"),
"소유자변경이력": insurance.get("ownerChangeCnt", "N/A"),
"내차피해금액": insurance.get("myAccidentCost", "N/A"),
"타차가해금액": insurance.get("otherAccidentCost", "N/A"),
"번호판변경이력": insurance.get("carNoChangeCnt", "N/A"),
"전체침수횟수": insurance.get("floodTotalLossCnt", "N/A"),
"일부침수횟수": insurance.get("floodPartLossCnt", "N/A"),
})
ins = pd.DataFrame(insurance_data)

#중복 제거
print(f"기존 행 수: {len(ins)}")
ins = ins.drop_duplicates()
print(f"중복 제거 후 행 수: {len(ins)}")


#기존 기본 정보의 금액 단위와 맞추기 위해 만원 단위로 변환
ins['내차피해금액'] = ins['내차피해금액']/10000
ins['타차가해금액'] = ins['타차가해금액']/10000


# csv 파일로 백업
ins.to_csv('팰리세이드_보험이력.csv', index=False, encoding='utf-8-sig')

# 차량번호 추가한 기본 정보에 보험 이력 추가
df_unique_no_ins = pd.merge(df_unique_no, ins, on='차량번호', how='inner')


df_unique_no_ins.to_csv('팰리세이드_기본_중복제거_차량번호_보험이력.csv', index=False, encoding='utf-8-sig')
print("All data has been saved to '팰리세이드_기본_중복제거_차량번호_보험이력.csv'")

## Encar 스타리아 자료 수집

In [ ]:
###### ENCAR 사이트에서 중고차 기본 정보 가져오기 - 스타리아 ######

import requests
import json
import time
import pandas as pd
from collections import Counter



#### 스타리아 기본 데이터 저장 ####
# Base URL 및 헤더 설정
base_url = "https://api.encar.com/search/car/list/premium"
headers = {
    "User-Agent": "Mozilla/5.0"
}

# 전체 데이터를 저장할 리스트
all_data = []
offset = 0
limit = 20

while True:
    # API 요청 파라미터 설정
    params = {
        "count": "true",
        "q": "(And.Hidden.N._.(C.CarType.Y._.(C.Manufacturer.현대._.ModelGroup.스타리아.)))",
        "sr": f"|ModifiedDate|{offset}|{limit}"
    }

    print(f"🚀 Fetching data from offset {offset}...")
    
    try:
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"🚨 요청 중 오류 발생: {e}")
        break
    except json.JSONDecodeError as e:
        print(f"🚨 JSON 파싱 오류 발생: {e}")
        break

    # SearchResults에서 데이터 추출
    results = data.get("SearchResults", [])
    
    if not results:  # 빈 데이터 처리 (마지막 페이지 도달)
        print("✅ 마지막 페이지 도달 또는 데이터 없음.")
        break

    # 전체 데이터 리스트에 추가
    all_data.extend(results)
    print(f"✅ Offset {offset}: {len(results)}건 추가 완료 (누적: {len(all_data)}건)")

    # 다음 offset으로 이동
    offset += limit

    # 서버 과부하 방지 (1초 대기)
    time.sleep(1)

# 결과를 JSON 파일로 저장
try:
    with open("스타리아_기본.json", "w", encoding="utf-8-sig") as f:
        json.dump(all_data, f, ensure_ascii=False, indent=4)
    print("✅ JSON 파일 저장 완료")
except Exception as e:
    print(f"🚨 파일 저장 중 오류 발생: {e}")

print(f"\n🎉 총 {len(all_data)}건의 스타리아 데이터 저장 완료!")







#### 저장된 json 파일에서 데이터 끌어와 데이터 프레임으로 만들기 ####

with open("스타리아_기본.json", "r", encoding="utf-8-sig") as file:
data = json.load(file)

# 필요한 정보만 추출
car_data = []
for car in data:
car_data.append({
"Id": car.get("Id", "N/A"),
"제조사": car.get("Manufacturer", "N/A"),
"모델명": car.get("Model", "N/A"),
"배지": car.get("Badge", "N/A"), # 트림 정보
"세부 배지": car.get("BadgeDetail", "N/A"), # 세부 트림
"연식": car.get("Year", "N/A"),
"모델연도": car.get("FormYear", "N/A"),
"가격": car.get("Price", "N/A"),
"주행거리": car.get("Mileage", "N/A"),
"연료": car.get("FuelType", "N/A"),
"변속기": car.get("Transmission", "N/A"),
"등록 상태": car.get("ServiceCopyCar", "N/A"), # 중복 차량 여부
"판매 유형": car.get("SellType", "N/A"),
"마지막 수정일": car.get("ModifiedDate", "N/A"),
"판매지역": car.get("OfficeCityState", "N/A"),
"판매처": car.get("OfficeName", "N/A"),
})

df = pd.DataFrame(car_data)


### 등록상태 DUPLICATION 날리기 ###
df_unique = df.loc[df['등록 상태']!='DUPLICATION']


### 기본 정보 안에서 Id 중복 제거 ###
df_unique = df_unique.drop_duplicates(subset='Id')
df_unique.to_csv('스타리아_기본_중복제거.csv', index=False, encoding='utf-8-sig')






#### 기존 데이터 중 Id로 다른 API를 통해 차량 번호 가져온 후 새로운 열로 추가 ####


# CSV 파일 불러오기
df_unique = pd.read_csv('스타리아_기본_중복제거.csv')

# API 기본 URL 설정
base_url = "https://api.encar.com/v1/readside/vehicle/{Id}?include=CONTENTS"

# vehicleNo 저장할 빈 리스트 생성
vehicle_numbers = []

# 각 Id에 대해 API 호출 및 JSON 저장하기
for idx, car_id in enumerate(df_unique['Id'], start=1):
url = base_url.format(Id=car_id)
try:
response = requests.get(url)
response.raise_for_status()
data = response.json()

# vehicleNo 추출
vehicle_no = data.get('vehicleNo', None)
vehicle_numbers.append(vehicle_no)
print(f"Processed {idx}/{len(df_unique)}: ID={car_id}")

except requests.exceptions.RequestException as e:
print(f"Error fetching data for ID {car_id}: {e}")
vehicle_numbers.append(None)

# 서버 과부하 방지를 위한 지연 추가
time.sleep(0.5)

# 원본 DataFrame에 vehicleNo 열 추가하기
df_unique['차량번호'] = vehicle_numbers

# 결과를 새로운 CSV 파일로 저장
df_unique.to_csv('스타리아_기본_중복제거_차량번호.csv', index=False, encoding='utf-8-sig')






#### Id와 차량번호 이용해 보험이력 데이터 가져와 json 파일로 저장 ####


# 차량 번호 추가한 CSV 파일 불러오기
df_unique_no = pd.read_csv('스타리아_기본_중복제거_차량번호.csv')
df_unique_no = df_unique_no.rename(columns={'vehicleNo':'차량번호'})
# API 기본 URL 설정
base_url = "https://api.encar.com/v1/readside/record/vehicle/{Id}/open?vehicleNo={CarNo}"

# JSON 데이터를 저장할 리스트 생성
all_data = []

# 각 Id와 vehicleNo에 대해 API 호출 및 데이터 수집
for idx, row in df_unique_no.iterrows():
car_id = row['Id']
vehicle_no = row['차량번호']
url = base_url.format(Id=car_id, CarNo=vehicle_no)

try:
response = requests.get(url)
response.raise_for_status()
data = response.json()

# JSON 데이터 리스트에 추가
all_data.append(data)

# 진행 상태 출력
print(f"Processed {idx + 1}/{len(df_unique_no)}: ID={car_id}, Vehicle No={vehicle_no}")

except requests.exceptions.RequestException as e:
print(f"Error fetching data for ID {car_id}, Vehicle No {vehicle_no}: {e}")

# 서버 과부하 방지를 위한 지연 추가
time.sleep(0.5)

# 모든 데이터를 하나의 JSON 파일로 저장
with open("스타리아_기본_중복제거_차량번호_보험이력.json", "w", encoding='utf-8-sig') as json_file:
json.dump(all_data, json_file, ensure_ascii=False, indent=4)

print("All data has been saved to '스타리아_기본_중복제거_차량번호_보험이력.json'")








#### json데이터에서 원하는 보험이력 정보 선택해 데이터프레임 만들기 ####

with open("스타리아_기본_중복제거_차량번호_보험이력.json", "r", encoding="utf-8-sig") as file:
data = json.load(file)

# 필요한 정보만 추출
insurance_data = []
for insurance in data:
insurance_data.append({
"차량번호": insurance.get("carNo", "N/A"),
"내차피해": insurance.get("myAccidentCnt", "N/A"),
"타차가해": insurance.get("otherAccidentCnt", "N/A"),
"소유자변경이력": insurance.get("ownerChangeCnt", "N/A"),
"내차피해금액": insurance.get("myAccidentCost", "N/A"),
"타차가해금액": insurance.get("otherAccidentCost", "N/A"),
"번호판변경이력": insurance.get("carNoChangeCnt", "N/A"),
"전체침수횟수": insurance.get("floodTotalLossCnt", "N/A"),
"일부침수횟수": insurance.get("floodPartLossCnt", "N/A"),
})
ins = pd.DataFrame(insurance_data)

#중복 제거
print(f"기존 행 수: {len(ins)}")
ins = ins.drop_duplicates()
print(f"중복 제거 후 행 수: {len(ins)}")


#기존 기본 정보의 금액 단위와 맞추기 위해 만원 단위로 변환
ins['내차피해금액'] = ins['내차피해금액']/10000
ins['타차가해금액'] = ins['타차가해금액']/10000


# csv 파일로 백업
ins.to_csv('스타리아_보험이력.csv', index=False, encoding='utf-8-sig')

# 차량번호 추가한 기본 정보에 보험 이력 추가
df_unique_no_ins = pd.merge(df_unique_no, ins, on='차량번호', how='inner')


df_unique_no_ins.to_csv('스타리아_기본_중복제거_차량번호_보험이력.csv', index=False, encoding='utf-8-sig')
print("All data has been saved to '스타리아_기본_중복제거_차량번호_보험이력.csv'")

## Encar 싼타페 자료 수집

In [ ]:
###### ENCAR 사이트에서 중고차 기본 정보 가져오기 - 싼타페 ######

import requests
import json
import time
import pandas as pd
from collections import Counter



#### 싼타페 기본 데이터 저장 ####
# Base URL 및 헤더 설정
base_url = "https://api.encar.com/search/car/list/premium"
headers = {
    "User-Agent": "Mozilla/5.0"
}

# 전체 데이터를 저장할 리스트
all_data = []
offset = 0
limit = 20

while True:
    # API 요청 파라미터 설정
    params = {
        "count": "true",
        "q": "(And.Hidden.N._.(C.CarType.Y._.(C.Manufacturer.현대._.ModelGroup.싼타페.)))",
        "sr": f"|ModifiedDate|{offset}|{limit}"
    }

    print(f"🚀 Fetching data from offset {offset}...")
    
    try:
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"🚨 요청 중 오류 발생: {e}")
        break
    except json.JSONDecodeError as e:
        print(f"🚨 JSON 파싱 오류 발생: {e}")
        break

    # SearchResults에서 데이터 추출
    results = data.get("SearchResults", [])
    
    if not results:  # 빈 데이터 처리 (마지막 페이지 도달)
        print("✅ 마지막 페이지 도달 또는 데이터 없음.")
        break

    # 전체 데이터 리스트에 추가
    all_data.extend(results)
    print(f"✅ Offset {offset}: {len(results)}건 추가 완료 (누적: {len(all_data)}건)")

    # 다음 offset으로 이동
    offset += limit

    # 서버 과부하 방지 (1초 대기)
    time.sleep(1)

# 결과를 JSON 파일로 저장
try:
    with open("싼타페_기본.json", "w", encoding="utf-8-sig") as f:
        json.dump(all_data, f, ensure_ascii=False, indent=4)
    print("✅ JSON 파일 저장 완료")
except Exception as e:
    print(f"🚨 파일 저장 중 오류 발생: {e}")

print(f"\n🎉 총 {len(all_data)}건의 싼타페 데이터 저장 완료!")







#### 저장된 json 파일에서 데이터 끌어와 데이터 프레임으로 만들기 ####

with open("싼타페_기본.json", "r", encoding="utf-8-sig") as file:
data = json.load(file)

# 필요한 정보만 추출
car_data = []
for car in data:
car_data.append({
"Id": car.get("Id", "N/A"),
"제조사": car.get("Manufacturer", "N/A"),
"모델명": car.get("Model", "N/A"),
"배지": car.get("Badge", "N/A"), # 트림 정보
"세부 배지": car.get("BadgeDetail", "N/A"), # 세부 트림
"연식": car.get("Year", "N/A"),
"모델연도": car.get("FormYear", "N/A"),
"가격": car.get("Price", "N/A"),
"주행거리": car.get("Mileage", "N/A"),
"연료": car.get("FuelType", "N/A"),
"변속기": car.get("Transmission", "N/A"),
"등록 상태": car.get("ServiceCopyCar", "N/A"), # 중복 차량 여부
"판매 유형": car.get("SellType", "N/A"),
"마지막 수정일": car.get("ModifiedDate", "N/A"),
"판매지역": car.get("OfficeCityState", "N/A"),
"판매처": car.get("OfficeName", "N/A"),
})

df = pd.DataFrame(car_data)


### 등록상태 DUPLICATION 날리기 ###
df_unique = df.loc[df['등록 상태']!='DUPLICATION']


### 기본 정보 안에서 Id 중복 제거 ###
df_unique = df_unique.drop_duplicates(subset='Id')
df_unique.to_csv('싼타페_기본_중복제거.csv', index=False, encoding='utf-8-sig')






#### 기존 데이터 중 Id로 다른 API를 통해 차량 번호 가져온 후 새로운 열로 추가 ####


# CSV 파일 불러오기
df_unique = pd.read_csv('싼타페_기본_중복제거.csv')

# API 기본 URL 설정
base_url = "https://api.encar.com/v1/readside/vehicle/{Id}?include=CONTENTS"

# vehicleNo 저장할 빈 리스트 생성
vehicle_numbers = []

# 각 Id에 대해 API 호출 및 JSON 저장하기
for idx, car_id in enumerate(df_unique['Id'], start=1):
url = base_url.format(Id=car_id)
try:
response = requests.get(url)
response.raise_for_status()
data = response.json()

# vehicleNo 추출
vehicle_no = data.get('vehicleNo', None)
vehicle_numbers.append(vehicle_no)
print(f"Processed {idx}/{len(df_unique)}: ID={car_id}")

except requests.exceptions.RequestException as e:
print(f"Error fetching data for ID {car_id}: {e}")
vehicle_numbers.append(None)

# 서버 과부하 방지를 위한 지연 추가
time.sleep(0.5)

# 원본 DataFrame에 vehicleNo 열 추가하기
df_unique['차량번호'] = vehicle_numbers

# 결과를 새로운 CSV 파일로 저장
df_unique.to_csv('싼타페_기본_중복제거_차량번호.csv', index=False, encoding='utf-8-sig')






#### Id와 차량번호 이용해 보험이력 데이터 가져와 json 파일로 저장 ####


# 차량 번호 추가한 CSV 파일 불러오기
df_unique_no = pd.read_csv('싼타페_기본_중복제거_차량번호.csv')
df_unique_no = df_unique_no.rename(columns={'vehicleNo':'차량번호'})
# API 기본 URL 설정
base_url = "https://api.encar.com/v1/readside/record/vehicle/{Id}/open?vehicleNo={CarNo}"

# JSON 데이터를 저장할 리스트 생성
all_data = []

# 각 Id와 vehicleNo에 대해 API 호출 및 데이터 수집
for idx, row in df_unique_no.iterrows():
car_id = row['Id']
vehicle_no = row['차량번호']
url = base_url.format(Id=car_id, CarNo=vehicle_no)

try:
response = requests.get(url)
response.raise_for_status()
data = response.json()

# JSON 데이터 리스트에 추가
all_data.append(data)

# 진행 상태 출력
print(f"Processed {idx + 1}/{len(df_unique_no)}: ID={car_id}, Vehicle No={vehicle_no}")

except requests.exceptions.RequestException as e:
print(f"Error fetching data for ID {car_id}, Vehicle No {vehicle_no}: {e}")

# 서버 과부하 방지를 위한 지연 추가
time.sleep(0.5)

# 모든 데이터를 하나의 JSON 파일로 저장
with open("싼타페_기본_중복제거_차량번호_보험이력.json", "w", encoding='utf-8-sig') as json_file:
json.dump(all_data, json_file, ensure_ascii=False, indent=4)

print("All data has been saved to '싼타페_기본_중복제거_차량번호_보험이력.json'")








#### json데이터에서 원하는 보험이력 정보 선택해 데이터프레임 만들기 ####

with open("싼타페_기본_중복제거_차량번호_보험이력.json", "r", encoding="utf-8-sig") as file:
data = json.load(file)

# 필요한 정보만 추출
insurance_data = []
for insurance in data:
insurance_data.append({
"차량번호": insurance.get("carNo", "N/A"),
"내차피해": insurance.get("myAccidentCnt", "N/A"),
"타차가해": insurance.get("otherAccidentCnt", "N/A"),
"소유자변경이력": insurance.get("ownerChangeCnt", "N/A"),
"내차피해금액": insurance.get("myAccidentCost", "N/A"),
"타차가해금액": insurance.get("otherAccidentCost", "N/A"),
"번호판변경이력": insurance.get("carNoChangeCnt", "N/A"),
"전체침수횟수": insurance.get("floodTotalLossCnt", "N/A"),
"일부침수횟수": insurance.get("floodPartLossCnt", "N/A"),
})
ins = pd.DataFrame(insurance_data)

#중복 제거
print(f"기존 행 수: {len(ins)}")
ins = ins.drop_duplicates()
print(f"중복 제거 후 행 수: {len(ins)}")


#기존 기본 정보의 금액 단위와 맞추기 위해 만원 단위로 변환
ins['내차피해금액'] = ins['내차피해금액']/10000
ins['타차가해금액'] = ins['타차가해금액']/10000


# csv 파일로 백업
ins.to_csv('싼타페_보험이력.csv', index=False, encoding='utf-8-sig')

# 차량번호 추가한 기본 정보에 보험 이력 추가
df_unique_no_ins = pd.merge(df_unique_no, ins, on='차량번호', how='inner')


df_unique_no_ins.to_csv('싼타페_기본_중복제거_차량번호_보험이력.csv', index=False, encoding='utf-8-sig')
print("All data has been saved to '싼타페_기본_중복제거_차량번호_보험이력.csv'")

## Encar 코나 자료 수집

In [ ]:
###### ENCAR 사이트에서 중고차 기본 정보 가져오기 - 코나 ######

import requests
import json
import time
import pandas as pd
from collections import Counter



#### 코나 기본 데이터 저장 ####
# Base URL 및 헤더 설정
base_url = "https://api.encar.com/search/car/list/premium"
headers = {
    "User-Agent": "Mozilla/5.0"
}

# 전체 데이터를 저장할 리스트
all_data = []
offset = 0
limit = 20

while True:
    # API 요청 파라미터 설정
    params = {
        "count": "true",
        "q": "(And.Hidden.N._.(C.CarType.Y._.(C.Manufacturer.현대._.ModelGroup.코나.)))",
        "sr": f"|ModifiedDate|{offset}|{limit}"
    }

    print(f"🚀 Fetching data from offset {offset}...")
    
    try:
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"🚨 요청 중 오류 발생: {e}")
        break
    except json.JSONDecodeError as e:
        print(f"🚨 JSON 파싱 오류 발생: {e}")
        break

    # SearchResults에서 데이터 추출
    results = data.get("SearchResults", [])
    
    if not results:  # 빈 데이터 처리 (마지막 페이지 도달)
        print("✅ 마지막 페이지 도달 또는 데이터 없음.")
        break

    # 전체 데이터 리스트에 추가
    all_data.extend(results)
    print(f"✅ Offset {offset}: {len(results)}건 추가 완료 (누적: {len(all_data)}건)")

    # 다음 offset으로 이동
    offset += limit

    # 서버 과부하 방지 (1초 대기)
    time.sleep(1)

# 결과를 JSON 파일로 저장
try:
    with open("코나_기본.json", "w", encoding="utf-8-sig") as f:
        json.dump(all_data, f, ensure_ascii=False, indent=4)
    print("✅ JSON 파일 저장 완료")
except Exception as e:
    print(f"🚨 파일 저장 중 오류 발생: {e}")

print(f"\n🎉 총 {len(all_data)}건의 코나 데이터 저장 완료!")







#### 저장된 json 파일에서 데이터 끌어와 데이터 프레임으로 만들기 ####

with open("코나_기본.json", "r", encoding="utf-8-sig") as file:
data = json.load(file)

# 필요한 정보만 추출
car_data = []
for car in data:
car_data.append({
"Id": car.get("Id", "N/A"),
"제조사": car.get("Manufacturer", "N/A"),
"모델명": car.get("Model", "N/A"),
"배지": car.get("Badge", "N/A"), # 트림 정보
"세부 배지": car.get("BadgeDetail", "N/A"), # 세부 트림
"연식": car.get("Year", "N/A"),
"모델연도": car.get("FormYear", "N/A"),
"가격": car.get("Price", "N/A"),
"주행거리": car.get("Mileage", "N/A"),
"연료": car.get("FuelType", "N/A"),
"변속기": car.get("Transmission", "N/A"),
"등록 상태": car.get("ServiceCopyCar", "N/A"), # 중복 차량 여부
"판매 유형": car.get("SellType", "N/A"),
"마지막 수정일": car.get("ModifiedDate", "N/A"),
"판매지역": car.get("OfficeCityState", "N/A"),
"판매처": car.get("OfficeName", "N/A"),
})

df = pd.DataFrame(car_data)


### 등록상태 DUPLICATION 날리기 ###
df_unique = df.loc[df['등록 상태']!='DUPLICATION']


### 기본 정보 안에서 Id 중복 제거 ###
df_unique = df_unique.drop_duplicates(subset='Id')
df_unique.to_csv('코나_기본_중복제거.csv', index=False, encoding='utf-8-sig')






#### 기존 데이터 중 Id로 다른 API를 통해 차량 번호 가져온 후 새로운 열로 추가 ####


# CSV 파일 불러오기
df_unique = pd.read_csv('코나_기본_중복제거.csv')

# API 기본 URL 설정
base_url = "https://api.encar.com/v1/readside/vehicle/{Id}?include=CONTENTS"

# vehicleNo 저장할 빈 리스트 생성
vehicle_numbers = []

# 각 Id에 대해 API 호출 및 JSON 저장하기
for idx, car_id in enumerate(df_unique['Id'], start=1):
url = base_url.format(Id=car_id)
try:
response = requests.get(url)
response.raise_for_status()
data = response.json()

# vehicleNo 추출
vehicle_no = data.get('vehicleNo', None)
vehicle_numbers.append(vehicle_no)
print(f"Processed {idx}/{len(df_unique)}: ID={car_id}")

except requests.exceptions.RequestException as e:
print(f"Error fetching data for ID {car_id}: {e}")
vehicle_numbers.append(None)

# 서버 과부하 방지를 위한 지연 추가
time.sleep(0.5)

# 원본 DataFrame에 vehicleNo 열 추가하기
df_unique['차량번호'] = vehicle_numbers

# 결과를 새로운 CSV 파일로 저장
df_unique.to_csv('코나_기본_중복제거_차량번호.csv', index=False, encoding='utf-8-sig')






#### Id와 차량번호 이용해 보험이력 데이터 가져와 json 파일로 저장 ####


# 차량 번호 추가한 CSV 파일 불러오기
df_unique_no = pd.read_csv('코나_기본_중복제거_차량번호.csv')
df_unique_no = df_unique_no.rename(columns={'vehicleNo':'차량번호'})
# API 기본 URL 설정
base_url = "https://api.encar.com/v1/readside/record/vehicle/{Id}/open?vehicleNo={CarNo}"

# JSON 데이터를 저장할 리스트 생성
all_data = []

# 각 Id와 vehicleNo에 대해 API 호출 및 데이터 수집
for idx, row in df_unique_no.iterrows():
car_id = row['Id']
vehicle_no = row['차량번호']
url = base_url.format(Id=car_id, CarNo=vehicle_no)

try:
response = requests.get(url)
response.raise_for_status()
data = response.json()

# JSON 데이터 리스트에 추가
all_data.append(data)

# 진행 상태 출력
print(f"Processed {idx + 1}/{len(df_unique_no)}: ID={car_id}, Vehicle No={vehicle_no}")

except requests.exceptions.RequestException as e:
print(f"Error fetching data for ID {car_id}, Vehicle No {vehicle_no}: {e}")

# 서버 과부하 방지를 위한 지연 추가
time.sleep(0.5)

# 모든 데이터를 하나의 JSON 파일로 저장
with open("코나_기본_중복제거_차량번호_보험이력.json", "w", encoding='utf-8-sig') as json_file:
json.dump(all_data, json_file, ensure_ascii=False, indent=4)

print("All data has been saved to '코나_기본_중복제거_차량번호_보험이력.json'")








#### json데이터에서 원하는 보험이력 정보 선택해 데이터프레임 만들기 ####

with open("코나_기본_중복제거_차량번호_보험이력.json", "r", encoding="utf-8-sig") as file:
data = json.load(file)

# 필요한 정보만 추출
insurance_data = []
for insurance in data:
insurance_data.append({
"차량번호": insurance.get("carNo", "N/A"),
"내차피해": insurance.get("myAccidentCnt", "N/A"),
"타차가해": insurance.get("otherAccidentCnt", "N/A"),
"소유자변경이력": insurance.get("ownerChangeCnt", "N/A"),
"내차피해금액": insurance.get("myAccidentCost", "N/A"),
"타차가해금액": insurance.get("otherAccidentCost", "N/A"),
"번호판변경이력": insurance.get("carNoChangeCnt", "N/A"),
"전체침수횟수": insurance.get("floodTotalLossCnt", "N/A"),
"일부침수횟수": insurance.get("floodPartLossCnt", "N/A"),
})
ins = pd.DataFrame(insurance_data)

#중복 제거
print(f"기존 행 수: {len(ins)}")
ins = ins.drop_duplicates()
print(f"중복 제거 후 행 수: {len(ins)}")


#기존 기본 정보의 금액 단위와 맞추기 위해 만원 단위로 변환
ins['내차피해금액'] = ins['내차피해금액']/10000
ins['타차가해금액'] = ins['타차가해금액']/10000


# csv 파일로 백업
ins.to_csv('코나_보험이력.csv', index=False, encoding='utf-8-sig')

# 차량번호 추가한 기본 정보에 보험 이력 추가
df_unique_no_ins = pd.merge(df_unique_no, ins, on='차량번호', how='inner')


df_unique_no_ins.to_csv('코나_기본_중복제거_차량번호_보험이력.csv', index=False, encoding='utf-8-sig')
print("All data has been saved to '코나_기본_중복제거_차량번호_보험이력.csv'")

## Encar 스타렉스 자료 수집

In [ ]:
###### ENCAR 사이트에서 중고차 기본 정보 가져오기 - 스타렉스 ######

import requests
import json
import time
import pandas as pd
from collections import Counter



#### 스타렉스 기본 데이터 저장 ####
# Base URL 및 헤더 설정
base_url = "https://api.encar.com/search/car/list/premium"
headers = {
    "User-Agent": "Mozilla/5.0"
}

# 전체 데이터를 저장할 리스트
all_data = []
offset = 0
limit = 20

while True:
    # API 요청 파라미터 설정
    params = {
        "count": "true",
        "q": "(And.Hidden.N._.(C.CarType.Y._.(C.Manufacturer.현대._.ModelGroup.스타렉스.)))",
        "sr": f"|ModifiedDate|{offset}|{limit}"
    }

    print(f"🚀 Fetching data from offset {offset}...")
    
    try:
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"🚨 요청 중 오류 발생: {e}")
        break
    except json.JSONDecodeError as e:
        print(f"🚨 JSON 파싱 오류 발생: {e}")
        break

    # SearchResults에서 데이터 추출
    results = data.get("SearchResults", [])
    
    if not results:  # 빈 데이터 처리 (마지막 페이지 도달)
        print("✅ 마지막 페이지 도달 또는 데이터 없음.")
        break

    # 전체 데이터 리스트에 추가
    all_data.extend(results)
    print(f"✅ Offset {offset}: {len(results)}건 추가 완료 (누적: {len(all_data)}건)")

    # 다음 offset으로 이동
    offset += limit

    # 서버 과부하 방지 (1초 대기)
    time.sleep(1)

# 결과를 JSON 파일로 저장
try:
    with open("스타렉스_기본.json", "w", encoding="utf-8-sig") as f:
        json.dump(all_data, f, ensure_ascii=False, indent=4)
    print("✅ JSON 파일 저장 완료")
except Exception as e:
    print(f"🚨 파일 저장 중 오류 발생: {e}")

print(f"\n🎉 총 {len(all_data)}건의 스타렉스 데이터 저장 완료!")







#### 저장된 json 파일에서 데이터 끌어와 데이터 프레임으로 만들기 ####

with open("스타렉스_기본.json", "r", encoding="utf-8-sig") as file:
data = json.load(file)

# 필요한 정보만 추출
car_data = []
for car in data:
car_data.append({
"Id": car.get("Id", "N/A"),
"제조사": car.get("Manufacturer", "N/A"),
"모델명": car.get("Model", "N/A"),
"배지": car.get("Badge", "N/A"), # 트림 정보
"세부 배지": car.get("BadgeDetail", "N/A"), # 세부 트림
"연식": car.get("Year", "N/A"),
"모델연도": car.get("FormYear", "N/A"),
"가격": car.get("Price", "N/A"),
"주행거리": car.get("Mileage", "N/A"),
"연료": car.get("FuelType", "N/A"),
"변속기": car.get("Transmission", "N/A"),
"등록 상태": car.get("ServiceCopyCar", "N/A"), # 중복 차량 여부
"판매 유형": car.get("SellType", "N/A"),
"마지막 수정일": car.get("ModifiedDate", "N/A"),
"판매지역": car.get("OfficeCityState", "N/A"),
"판매처": car.get("OfficeName", "N/A"),
})

df = pd.DataFrame(car_data)


### 등록상태 DUPLICATION 날리기 ###
df_unique = df.loc[df['등록 상태']!='DUPLICATION']


### 기본 정보 안에서 Id 중복 제거 ###
df_unique = df_unique.drop_duplicates(subset='Id')
df_unique.to_csv('스타렉스_기본_중복제거.csv', index=False, encoding='utf-8-sig')






#### 기존 데이터 중 Id로 다른 API를 통해 차량 번호 가져온 후 새로운 열로 추가 ####


# CSV 파일 불러오기
df_unique = pd.read_csv('스타렉스_기본_중복제거.csv')

# API 기본 URL 설정
base_url = "https://api.encar.com/v1/readside/vehicle/{Id}?include=CONTENTS"

# vehicleNo 저장할 빈 리스트 생성
vehicle_numbers = []

# 각 Id에 대해 API 호출 및 JSON 저장하기
for idx, car_id in enumerate(df_unique['Id'], start=1):
url = base_url.format(Id=car_id)
try:
response = requests.get(url)
response.raise_for_status()
data = response.json()

# vehicleNo 추출
vehicle_no = data.get('vehicleNo', None)
vehicle_numbers.append(vehicle_no)
print(f"Processed {idx}/{len(df_unique)}: ID={car_id}")

except requests.exceptions.RequestException as e:
print(f"Error fetching data for ID {car_id}: {e}")
vehicle_numbers.append(None)

# 서버 과부하 방지를 위한 지연 추가
time.sleep(0.5)

# 원본 DataFrame에 vehicleNo 열 추가하기
df_unique['차량번호'] = vehicle_numbers

# 결과를 새로운 CSV 파일로 저장
df_unique.to_csv('스타렉스_기본_중복제거_차량번호.csv', index=False, encoding='utf-8-sig')






#### Id와 차량번호 이용해 보험이력 데이터 가져와 json 파일로 저장 ####


# 차량 번호 추가한 CSV 파일 불러오기
df_unique_no = pd.read_csv('스타렉스_기본_중복제거_차량번호.csv')
df_unique_no = df_unique_no.rename(columns={'vehicleNo':'차량번호'})
# API 기본 URL 설정
base_url = "https://api.encar.com/v1/readside/record/vehicle/{Id}/open?vehicleNo={CarNo}"

# JSON 데이터를 저장할 리스트 생성
all_data = []

# 각 Id와 vehicleNo에 대해 API 호출 및 데이터 수집
for idx, row in df_unique_no.iterrows():
car_id = row['Id']
vehicle_no = row['차량번호']
url = base_url.format(Id=car_id, CarNo=vehicle_no)

try:
response = requests.get(url)
response.raise_for_status()
data = response.json()

# JSON 데이터 리스트에 추가
all_data.append(data)

# 진행 상태 출력
print(f"Processed {idx + 1}/{len(df_unique_no)}: ID={car_id}, Vehicle No={vehicle_no}")

except requests.exceptions.RequestException as e:
print(f"Error fetching data for ID {car_id}, Vehicle No {vehicle_no}: {e}")

# 서버 과부하 방지를 위한 지연 추가
time.sleep(0.5)

# 모든 데이터를 하나의 JSON 파일로 저장
with open("스타렉스_기본_중복제거_차량번호_보험이력.json", "w", encoding='utf-8-sig') as json_file:
json.dump(all_data, json_file, ensure_ascii=False, indent=4)

print("All data has been saved to '스타렉스_기본_중복제거_차량번호_보험이력.json'")








#### json데이터에서 원하는 보험이력 정보 선택해 데이터프레임 만들기 ####

with open("스타렉스_기본_중복제거_차량번호_보험이력.json", "r", encoding="utf-8-sig") as file:
data = json.load(file)

# 필요한 정보만 추출
insurance_data = []
for insurance in data:
insurance_data.append({
"차량번호": insurance.get("carNo", "N/A"),
"내차피해": insurance.get("myAccidentCnt", "N/A"),
"타차가해": insurance.get("otherAccidentCnt", "N/A"),
"소유자변경이력": insurance.get("ownerChangeCnt", "N/A"),
"내차피해금액": insurance.get("myAccidentCost", "N/A"),
"타차가해금액": insurance.get("otherAccidentCost", "N/A"),
"번호판변경이력": insurance.get("carNoChangeCnt", "N/A"),
"전체침수횟수": insurance.get("floodTotalLossCnt", "N/A"),
"일부침수횟수": insurance.get("floodPartLossCnt", "N/A"),
})
ins = pd.DataFrame(insurance_data)

#중복 제거
print(f"기존 행 수: {len(ins)}")
ins = ins.drop_duplicates()
print(f"중복 제거 후 행 수: {len(ins)}")


#기존 기본 정보의 금액 단위와 맞추기 위해 만원 단위로 변환
ins['내차피해금액'] = ins['내차피해금액']/10000
ins['타차가해금액'] = ins['타차가해금액']/10000


# csv 파일로 백업
ins.to_csv('스타렉스_보험이력.csv', index=False, encoding='utf-8-sig')

# 차량번호 추가한 기본 정보에 보험 이력 추가
df_unique_no_ins = pd.merge(df_unique_no, ins, on='차량번호', how='inner')


df_unique_no_ins.to_csv('스타렉스_기본_중복제거_차량번호_보험이력.csv', index=False, encoding='utf-8-sig')
print("All data has been saved to '스타렉스_기본_중복제거_차량번호_보험이력.csv'")

## Encar 그랜저 자료 수집

In [ ]:
###### ENCAR 사이트에서 중고차 기본 정보 가져오기 - 그랜저 ######

import requests
import json
import time
import pandas as pd
from collections import Counter



#### 그랜저 기본 데이터 저장 ####
# Base URL 및 헤더 설정
base_url = "https://api.encar.com/search/car/list/premium"
headers = {
    "User-Agent": "Mozilla/5.0"
}

# 전체 데이터를 저장할 리스트
all_data = []
offset = 0
limit = 20

while True:
    # API 요청 파라미터 설정
    params = {
        "count": "true",
        "q": "(And.Hidden.N._.(C.CarType.Y._.(C.Manufacturer.현대._.ModelGroup.그랜저.)))",
        "sr": f"|ModifiedDate|{offset}|{limit}"
    }

    print(f"🚀 Fetching data from offset {offset}...")
    
    try:
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"🚨 요청 중 오류 발생: {e}")
        break
    except json.JSONDecodeError as e:
        print(f"🚨 JSON 파싱 오류 발생: {e}")
        break

    # SearchResults에서 데이터 추출
    results = data.get("SearchResults", [])
    
    if not results:  # 빈 데이터 처리 (마지막 페이지 도달)
        print("✅ 마지막 페이지 도달 또는 데이터 없음.")
        break

    # 전체 데이터 리스트에 추가
    all_data.extend(results)
    print(f"✅ Offset {offset}: {len(results)}건 추가 완료 (누적: {len(all_data)}건)")

    # 다음 offset으로 이동
    offset += limit

    # 서버 과부하 방지 (1초 대기)
    time.sleep(1)

# 결과를 JSON 파일로 저장
try:
    with open("그랜저_기본.json", "w", encoding="utf-8-sig") as f:
        json.dump(all_data, f, ensure_ascii=False, indent=4)
    print("✅ JSON 파일 저장 완료")
except Exception as e:
    print(f"🚨 파일 저장 중 오류 발생: {e}")

print(f"\n🎉 총 {len(all_data)}건의 그랜저 데이터 저장 완료!")







#### 저장된 json 파일에서 데이터 끌어와 데이터 프레임으로 만들기 ####

with open("그랜저_기본.json", "r", encoding="utf-8-sig") as file:
data = json.load(file)

# 필요한 정보만 추출
car_data = []
for car in data:
car_data.append({
"Id": car.get("Id", "N/A"),
"제조사": car.get("Manufacturer", "N/A"),
"모델명": car.get("Model", "N/A"),
"배지": car.get("Badge", "N/A"), # 트림 정보
"세부 배지": car.get("BadgeDetail", "N/A"), # 세부 트림
"연식": car.get("Year", "N/A"),
"모델연도": car.get("FormYear", "N/A"),
"가격": car.get("Price", "N/A"),
"주행거리": car.get("Mileage", "N/A"),
"연료": car.get("FuelType", "N/A"),
"변속기": car.get("Transmission", "N/A"),
"등록 상태": car.get("ServiceCopyCar", "N/A"), # 중복 차량 여부
"판매 유형": car.get("SellType", "N/A"),
"마지막 수정일": car.get("ModifiedDate", "N/A"),
"판매지역": car.get("OfficeCityState", "N/A"),
"판매처": car.get("OfficeName", "N/A"),
})

df = pd.DataFrame(car_data)


### 등록상태 DUPLICATION 날리기 ###
df_unique = df.loc[df['등록 상태']!='DUPLICATION']


### 기본 정보 안에서 Id 중복 제거 ###
df_unique = df_unique.drop_duplicates(subset='Id')
df_unique.to_csv('그랜저_기본_중복제거.csv', index=False, encoding='utf-8-sig')






#### 기존 데이터 중 Id로 다른 API를 통해 차량 번호 가져온 후 새로운 열로 추가 ####


# CSV 파일 불러오기
df_unique = pd.read_csv('그랜저_기본_중복제거.csv')

# API 기본 URL 설정
base_url = "https://api.encar.com/v1/readside/vehicle/{Id}?include=CONTENTS"

# vehicleNo 저장할 빈 리스트 생성
vehicle_numbers = []

# 각 Id에 대해 API 호출 및 JSON 저장하기
for idx, car_id in enumerate(df_unique['Id'], start=1):
url = base_url.format(Id=car_id)
try:
response = requests.get(url)
response.raise_for_status()
data = response.json()

# vehicleNo 추출
vehicle_no = data.get('vehicleNo', None)
vehicle_numbers.append(vehicle_no)
print(f"Processed {idx}/{len(df_unique)}: ID={car_id}")

except requests.exceptions.RequestException as e:
print(f"Error fetching data for ID {car_id}: {e}")
vehicle_numbers.append(None)

# 서버 과부하 방지를 위한 지연 추가
time.sleep(0.5)

# 원본 DataFrame에 vehicleNo 열 추가하기
df_unique['차량번호'] = vehicle_numbers

# 결과를 새로운 CSV 파일로 저장
df_unique.to_csv('그랜저_기본_중복제거_차량번호.csv', index=False, encoding='utf-8-sig')






#### Id와 차량번호 이용해 보험이력 데이터 가져와 json 파일로 저장 ####


# 차량 번호 추가한 CSV 파일 불러오기
df_unique_no = pd.read_csv('그랜저_기본_중복제거_차량번호.csv')
df_unique_no = df_unique_no.rename(columns={'vehicleNo':'차량번호'})
# API 기본 URL 설정
base_url = "https://api.encar.com/v1/readside/record/vehicle/{Id}/open?vehicleNo={CarNo}"

# JSON 데이터를 저장할 리스트 생성
all_data = []

# 각 Id와 vehicleNo에 대해 API 호출 및 데이터 수집
for idx, row in df_unique_no.iterrows():
car_id = row['Id']
vehicle_no = row['차량번호']
url = base_url.format(Id=car_id, CarNo=vehicle_no)

try:
response = requests.get(url)
response.raise_for_status()
data = response.json()

# JSON 데이터 리스트에 추가
all_data.append(data)

# 진행 상태 출력
print(f"Processed {idx + 1}/{len(df_unique_no)}: ID={car_id}, Vehicle No={vehicle_no}")

except requests.exceptions.RequestException as e:
print(f"Error fetching data for ID {car_id}, Vehicle No {vehicle_no}: {e}")

# 서버 과부하 방지를 위한 지연 추가
time.sleep(0.5)

# 모든 데이터를 하나의 JSON 파일로 저장
with open("그랜저_기본_중복제거_차량번호_보험이력.json", "w", encoding='utf-8-sig') as json_file:
json.dump(all_data, json_file, ensure_ascii=False, indent=4)

print("All data has been saved to '그랜저_기본_중복제거_차량번호_보험이력.json'")








#### json데이터에서 원하는 보험이력 정보 선택해 데이터프레임 만들기 ####

with open("그랜저_기본_중복제거_차량번호_보험이력.json", "r", encoding="utf-8-sig") as file:
data = json.load(file)

# 필요한 정보만 추출
insurance_data = []
for insurance in data:
insurance_data.append({
"차량번호": insurance.get("carNo", "N/A"),
"내차피해": insurance.get("myAccidentCnt", "N/A"),
"타차가해": insurance.get("otherAccidentCnt", "N/A"),
"소유자변경이력": insurance.get("ownerChangeCnt", "N/A"),
"내차피해금액": insurance.get("myAccidentCost", "N/A"),
"타차가해금액": insurance.get("otherAccidentCost", "N/A"),
"번호판변경이력": insurance.get("carNoChangeCnt", "N/A"),
"전체침수횟수": insurance.get("floodTotalLossCnt", "N/A"),
"일부침수횟수": insurance.get("floodPartLossCnt", "N/A"),
})
ins = pd.DataFrame(insurance_data)

#중복 제거
print(f"기존 행 수: {len(ins)}")
ins = ins.drop_duplicates()
print(f"중복 제거 후 행 수: {len(ins)}")


#기존 기본 정보의 금액 단위와 맞추기 위해 만원 단위로 변환
ins['내차피해금액'] = ins['내차피해금액']/10000
ins['타차가해금액'] = ins['타차가해금액']/10000


# csv 파일로 백업
ins.to_csv('그랜저_보험이력.csv', index=False, encoding='utf-8-sig')

# 차량번호 추가한 기본 정보에 보험 이력 추가
df_unique_no_ins = pd.merge(df_unique_no, ins, on='차량번호', how='inner')


df_unique_no_ins.to_csv('그랜저_기본_중복제거_차량번호_보험이력.csv', index=False, encoding='utf-8-sig')
print("All data has been saved to '그랜저_기본_중복제거_차량번호_보험이력.csv'")

## Encar 제네시스 자료 수집
##### - 제네시스는 브랜드가 현대가 아닌 제네시스로 되어 있는 관계로 API 주소가 약간 다름

In [ ]:
###### ENCAR 사이트에서 중고차 기본 정보 가져오기 - 제네시스스 ######

import requests
import json
import time
import pandas as pd
from collections import Counter



#### 제네시스 기본 데이터 저장 ####
# Base URL 및 헤더 설정
base_url = "https://api.encar.com/search/car/list/premium"
headers = {
    "User-Agent": "Mozilla/5.0"
}

# 전체 데이터를 저장할 리스트
all_data = []
offset = 0
limit = 20

while True:
    # API 요청 파라미터 설정
    params = {
        "count": "true",
        "q": "(And.Hidden.N._.(C.CarType.Y._.Manufacturer.제네시스.))",
        "sr": f"|ModifiedDate|{offset}|{limit}"
    }

    print(f"🚀 Fetching data from offset {offset}...")
    
    try:
        response = requests.get(base_url, headers=headers, params=params)
        response.raise_for_status()
        data = response.json()
    except requests.exceptions.RequestException as e:
        print(f"🚨 요청 중 오류 발생: {e}")
        break
    except json.JSONDecodeError as e:
        print(f"🚨 JSON 파싱 오류 발생: {e}")
        break

    # SearchResults에서 데이터 추출
    results = data.get("SearchResults", [])
    
    if not results:  # 빈 데이터 처리 (마지막 페이지 도달)
        print("✅ 마지막 페이지 도달 또는 데이터 없음.")
        break

    # 전체 데이터 리스트에 추가
    all_data.extend(results)
    print(f"✅ Offset {offset}: {len(results)}건 추가 완료 (누적: {len(all_data)}건)")

    # 다음 offset으로 이동
    offset += limit

    # 서버 과부하 방지 (1초 대기)
    time.sleep(1)

# 결과를 JSON 파일로 저장
try:
    with open("제네시스_기본.json", "w", encoding="utf-8-sig") as f:
        json.dump(all_data, f, ensure_ascii=False, indent=4)
    print("✅ JSON 파일 저장 완료")
except Exception as e:
    print(f"🚨 파일 저장 중 오류 발생: {e}")

print(f"\n🎉 총 {len(all_data)}건의 제네시스 데이터 저장 완료!")







#### 저장된 json 파일에서 데이터 끌어와 데이터 프레임으로 만들기 ####

with open("제네시스_기본.json", "r", encoding="utf-8-sig") as file:
data = json.load(file)

# 필요한 정보만 추출
car_data = []
for car in data:
car_data.append({
"Id": car.get("Id", "N/A"),
"제조사": car.get("Manufacturer", "N/A"),
"모델명": car.get("Model", "N/A"),
"배지": car.get("Badge", "N/A"), # 트림 정보
"세부 배지": car.get("BadgeDetail", "N/A"), # 세부 트림
"연식": car.get("Year", "N/A"),
"모델연도": car.get("FormYear", "N/A"),
"가격": car.get("Price", "N/A"),
"주행거리": car.get("Mileage", "N/A"),
"연료": car.get("FuelType", "N/A"),
"변속기": car.get("Transmission", "N/A"),
"등록 상태": car.get("ServiceCopyCar", "N/A"), # 중복 차량 여부
"판매 유형": car.get("SellType", "N/A"),
"마지막 수정일": car.get("ModifiedDate", "N/A"),
"판매지역": car.get("OfficeCityState", "N/A"),
"판매처": car.get("OfficeName", "N/A"),
})

df = pd.DataFrame(car_data)


### 등록상태 DUPLICATION 날리기 ###
df_unique = df.loc[df['등록 상태']!='DUPLICATION']


### 기본 정보 안에서 Id 중복 제거 ###
df_unique = df_unique.drop_duplicates(subset='Id')
df_unique.to_csv('제네시스_기본_중복제거.csv', index=False, encoding='utf-8-sig')






#### 기존 데이터 중 Id로 다른 API를 통해 차량 번호 가져온 후 새로운 열로 추가 ####


# CSV 파일 불러오기
df_unique = pd.read_csv('제네시스_기본_중복제거.csv')

# API 기본 URL 설정
base_url = "https://api.encar.com/v1/readside/vehicle/{Id}?include=CONTENTS"

# vehicleNo 저장할 빈 리스트 생성
vehicle_numbers = []

# 각 Id에 대해 API 호출 및 JSON 저장하기
for idx, car_id in enumerate(df_unique['Id'], start=1):
url = base_url.format(Id=car_id)
try:
response = requests.get(url)
response.raise_for_status()
data = response.json()

# vehicleNo 추출
vehicle_no = data.get('vehicleNo', None)
vehicle_numbers.append(vehicle_no)
print(f"Processed {idx}/{len(df_unique)}: ID={car_id}")

except requests.exceptions.RequestException as e:
print(f"Error fetching data for ID {car_id}: {e}")
vehicle_numbers.append(None)

# 서버 과부하 방지를 위한 지연 추가
time.sleep(0.5)

# 원본 DataFrame에 vehicleNo 열 추가하기
df_unique['차량번호'] = vehicle_numbers

# 결과를 새로운 CSV 파일로 저장
df_unique.to_csv('제네시스_기본_중복제거_차량번호.csv', index=False, encoding='utf-8-sig')






#### Id와 차량번호 이용해 보험이력 데이터 가져와 json 파일로 저장 ####


# 차량 번호 추가한 CSV 파일 불러오기
df_unique_no = pd.read_csv('제네시스_기본_중복제거_차량번호.csv')
df_unique_no = df_unique_no.rename(columns={'vehicleNo':'차량번호'})
# API 기본 URL 설정
base_url = "https://api.encar.com/v1/readside/record/vehicle/{Id}/open?vehicleNo={CarNo}"

# JSON 데이터를 저장할 리스트 생성
all_data = []

# 각 Id와 vehicleNo에 대해 API 호출 및 데이터 수집
for idx, row in df_unique_no.iterrows():
car_id = row['Id']
vehicle_no = row['차량번호']
url = base_url.format(Id=car_id, CarNo=vehicle_no)

try:
response = requests.get(url)
response.raise_for_status()
data = response.json()

# JSON 데이터 리스트에 추가
all_data.append(data)

# 진행 상태 출력
print(f"Processed {idx + 1}/{len(df_unique_no)}: ID={car_id}, Vehicle No={vehicle_no}")

except requests.exceptions.RequestException as e:
print(f"Error fetching data for ID {car_id}, Vehicle No {vehicle_no}: {e}")

# 서버 과부하 방지를 위한 지연 추가
time.sleep(0.5)

# 모든 데이터를 하나의 JSON 파일로 저장
with open("제네시스_기본_중복제거_차량번호_보험이력.json", "w", encoding='utf-8-sig') as json_file:
json.dump(all_data, json_file, ensure_ascii=False, indent=4)

print("All data has been saved to '제네시스_기본_중복제거_차량번호_보험이력.json'")








#### json데이터에서 원하는 보험이력 정보 선택해 데이터프레임 만들기 ####

with open("제네시스_기본_중복제거_차량번호_보험이력.json", "r", encoding="utf-8-sig") as file:
data = json.load(file)

# 필요한 정보만 추출
insurance_data = []
for insurance in data:
insurance_data.append({
"차량번호": insurance.get("carNo", "N/A"),
"내차피해": insurance.get("myAccidentCnt", "N/A"),
"타차가해": insurance.get("otherAccidentCnt", "N/A"),
"소유자변경이력": insurance.get("ownerChangeCnt", "N/A"),
"내차피해금액": insurance.get("myAccidentCost", "N/A"),
"타차가해금액": insurance.get("otherAccidentCost", "N/A"),
"번호판변경이력": insurance.get("carNoChangeCnt", "N/A"),
"전체침수횟수": insurance.get("floodTotalLossCnt", "N/A"),
"일부침수횟수": insurance.get("floodPartLossCnt", "N/A"),
})
ins = pd.DataFrame(insurance_data)

#중복 제거
print(f"기존 행 수: {len(ins)}")
ins = ins.drop_duplicates()
print(f"중복 제거 후 행 수: {len(ins)}")


#기존 기본 정보의 금액 단위와 맞추기 위해 만원 단위로 변환
ins['내차피해금액'] = ins['내차피해금액']/10000
ins['타차가해금액'] = ins['타차가해금액']/10000


# csv 파일로 백업
ins.to_csv('제네시스_보험이력.csv', index=False, encoding='utf-8-sig')

# 차량번호 추가한 기본 정보에 보험 이력 추가
df_unique_no_ins = pd.merge(df_unique_no, ins, on='차량번호', how='inner')


df_unique_no_ins.to_csv('제네시스_기본_중복제거_차량번호_보험이력.csv', index=False, encoding='utf-8-sig')
print("All data has been saved to '제네시스_기본_중복제거_차량번호_보험이력.csv'")